# Apply pixel classifier on all images in a folder,  create corresponding mask

In [1]:
from scipy import ndimage
from skimage.morphology import binary_erosion
import sys

from sklearn.ensemble import RandomForestClassifier
import glob
import os
import pickle as pkl
from skimage import io
from csbdeep.utils import normalize
import numpy as np
import skimage
from skimage.morphology import square
import scipy
from skimage.segmentation import expand_labels
import matplotlib.pyplot as plt
import pandas as pd
sys.path.append(os.path.realpath('/Users/max/Desktop/Scripts_for_max/'))
from semantic_segmentation import init_VGG16_pyramid, fd_VGG16_pyramid
from magicgui import magicgui
import napari


/Users/max/opt/anaconda3/lib/python3.7/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [2]:
def spot_mask_from_labels(labels):
    '''takes label mask, shrinks objects and subtracts from original image.'''
    stim_width = 10
    footprint = np.ones((stim_width,stim_width))
    labels_b = labels>0
    labels_b_ero = binary_erosion(labels_b,footprint)
    labels_b_sub = np.logical_xor(labels_b,labels_b_ero)
    labels_b_sub = labels_b_sub.astype('uint8')
    labels_sub = np.multiply(labels_b_sub,labels)
    
    df = pd.DataFrame()
    props = skimage.measure.regionprops(labels)
    for prop in props[:]:
        df_spot = pd.DataFrame({'cell_label': [prop.label],'cell_x': [prop.centroid[0]], 'cell_y':[prop.centroid[1]], 'cell_area': [prop.area],'stim_width':[stim_width]})
        df = df.append(df_spot)

    return labels_b_sub,df

def frame_to_labels(frame):
    #f_h2b = frame[1,:,:].copy()
    f_h2b = frame[:,:].copy()
    f_h2b_scaled = normalize(f_h2b, 5,95)
    features = fd_VGG16_pyramid(f_h2b_scaled,models,shapes)
    to_predict = features.reshape(np.shape(features)[0]*np.shape(features)[1],np.shape(features)[2])
    prediction_list = clf.predict_proba(to_predict)[:,0]
    prediction_img = np.reshape(prediction_list, f_h2b_scaled.shape)
    #now run watershed on segmentaion, using no seed
    prediction_img = prediction_img>0.5
    prediction_img = skimage.morphology.erosion(prediction_img, square(5))
    prediction_img = skimage.morphology.remove_small_objects(prediction_img, min_size = 100**2)
    prediction_img = scipy.ndimage.morphology.binary_fill_holes(prediction_img)
        
    #labels = skimage.segmentation.watershed(-distance_img, mask = prediction_img)#, mask=prediction_img>0.5)
    labels = skimage.segmentation.watershed(~prediction_img, mask = prediction_img)

    return labels

### Import the classifier you want to use

In [3]:
file = open("/Users/max/Desktop/Scripts_for_max/classifier_max_2021-10-15_DLC1.pkl", "rb")
clf = pkl.load(file)
file.close()

In [4]:
file = open("/Users/max/Desktop/Scripts_for_max/classifier_max_2021-10-15_DLC1.pkl", "wb")
pkl.dump(clf, file)

file.close()

In [ ]:
ch1_flatfield=np.array(Image.open('/Volumes/imaging.data/Max/REF52/DLC_1/FRET/FRET_5/FRET_5/Flatfield_s1_/FRET/Flatfield_s1_w26TIRFFRETacceptor_t3.tif'))
ch2_flatfield=np.array(Image.open('/Volumes/imaging.data/Max/REF52/DLC_1/FRET/FRET_5/FRET_5/Flatfield_s1_/CFP/Flatfield_s1_w16TIRF-CFP_t2.tif'))
ch1='FRET'
ch2='CFP'

In [ ]:
def bg_calculation(image):
    hist, bins = np.histogram(image, bins=50)
    width = 0.7 * (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:]) / 2
#    plt.bar(center, hist, align='center', width = width)
#    plt.ylim(0,450000)
#    plt.xlim(0,750)
    bg = np.mean(bins[0:4])
    return bg, #plt.show(

# Actual background substraction
def background_sub(image, bg):
    global bg_mask
    bg_z_image = image - bg
    bg_mask = np.zeros_like(bg_z_image)
    bg_mask[bg_z_image > 0] = 1
    bg_z_image = bg_mask*bg_z_image
    bg_z_image = bg_z_image.astype(np.float32) 
    return bg_z_image

def bg_correct(image):
    bg_calc = bg_calculation(image)
    corr_im = background_sub(image,bg_calc[0])
    return corr_im

def image_ratioing(f, c, path, FRET):
    FRETimage=f
    CFPimage=c
    FRETratio=np.divide(FRETimage, CFPimage)
    FRETratiotif=Image.fromarray(FRETratio)
    outpath=os.path.join(path, 'ratio')
    createFolder(outpath)
    savestr=os.path.join(outpath,FRET.split('.')[0]+'_ratio'+'.TIF')
    FRETratiotif.save(savestr)
    print('saving: ', savestr)
    
def flatfield_preprocessing(img, flatfield):
    ch1image=np.array(Image.open(os.path.join(path, img)))
    ch1corrected=np.divide(ch1image, flatfield)
    bg_corrected=bg_correct(ch1corrected)
    plt.imshow(bg_corrected, cmap = 'gray')
    return bg_corrected
def flatfield_correction(path, ch1='FRET', ch2='CFP', timelapse=False):
   files=os.listdir(path)
   if timelapse == False:
       pattern = re.compile('(?P<Imagename>[^w]*).*(?P<Channel>FRET|CFP).*')
   if timelapse == True:       
       pattern = re.compile('(?P<Imagename>[^w]*).*(?P<Channel>FRET|CFP).*(?P<Timepoint>t[0-9]+).*')

   FRET_files=[i for i in files if ch1 in i]
   CFP_files=[i for i in files if ch2 in i] 
   for FRET in FRET_files:
        try:
            if timelapse==False:
                FRETname=re.search(pattern, FRET).group('Imagename')
            if timelapse == True:       
                FRETname=re.search(pattern, FRET).group('Imagename', 'Timepoint')

            FRET_corrected=flatfield_preprocessing(FRET, ch1_flatfield)
            for CFP in CFP_files:
                if timelapse==False:
                    CFPname=re.search(pattern, CFP).group('Imagename')
                    CFPname=re.search(pattern, CFP).group('Imagename', 'Timepoint')

                if CFPname==FRETname:
                    try:
                        CFP_corrected=flatfield_preprocessing(CFP, ch2_flatfield)
                        image_ratioing(FRET_corrected, CFP_corrected, path, FRET)
                        print('processing ', FRET)
                    except OSError as e:
                        print (e)     
            
        except OSError as e:
            print (e)

### Set input and output folders

In [5]:
input_folder = '/Volumes/imaging.data/Max/REF52/DLC_1/FRET/FRET_11_late/'
output_folder_segmentation = '/Volumes/imaging.data/Max/REF52/DLC_1/FRET/FRET_11_late/cp.out1/output/'
output_folder_mask = '/Volumes/imaging.data/Max/REF52/DLC_1/FRET/FRET_11_late/cp.out1/output/stim_mask/'

### Search images

In [6]:
filenames = sorted(glob.glob(os.path.join(input_folder,'*.TIF')))
print(f'Found {len(filenames)} input files.')
print(f'Last file: {filenames[-2]}')
raw = io.imread(filenames[-1])

Found 333 input files.
Last file: /Volumes/imaging.data/Max/REF52/DLC_1/FRET/FRET_11_late/flatfield_3_w26TIRFFRETacceptor.TIF


In [7]:
project_path = '/Volumes/imaging.data/Max/REF52/DLC_1/FRET/FRET_11_late/'
fov = 0
raw = io.imread(filenames[4])
frame = raw
norm_min = 5
norm_max = 95
f_h2b = frame[:,:].copy()
annotation_frame = normalize(f_h2b, norm_min,norm_max)
norm_min = 5
norm_max = 95
shapes = [(512,512),(64,64),(32,32)]
#shapes = [(540,540),(68,68),(32,32)]
models = init_VGG16_pyramid(shapes)

In [7]:
#create the classifier
clf = RandomForestClassifier(random_state=0)

# create feature extractor and apply 
#shapes = [(1024,1024),(512,512),(256,256),(128,128),(64,64)]
shapes = [(512,512),(64,64),(32,32)]
models = init_VGG16_pyramid(shapes)
features = fd_VGG16_pyramid(annotation_frame,models,shapes)


@magicgui(call_button='train classifier')
def clf_widget():
    #run everytime the train button is pressed
    #extract the labels
    annotations = np.array(labels_layer.data)
    X,y = annotations_to_tensor(features,annotations)
    clf.fit(X, y) #train the classifier on the new labels
    #now predict the rest of the image with the new clf
    to_predict = features.reshape(np.shape(features)[0]*np.shape(features)[1],np.shape(features)[2])
    prediction_list = clf.predict_proba(to_predict)[:,0]
    prediction_img = np.reshape(prediction_list, annotation_frame.shape)
    
    #display the new prediciton
    prediction_layer.visible = True
    prediction_layer.data = prediction_img
    return clf

def annotations_to_tensor(feature_matrix,mask):
    #feature matrix dim: [x,y,nb_features]
    #possible mask elements: 0: not annotated, int[1,2]: class annotation
    y_labels=[] #where class labels are stored
    X_features=[] #where feature vectors are stored
    for x,y in np.argwhere(mask!=0):
        y_labels.append(mask[x,y])
        #X_features.append(feature_matrix[x,y,:])
        X_features.append(feature_matrix[x,y,])
    #turn list into np array
    X_features = np.asarray(X_features)
    return X_features,y_labels

viewer = napari.view_image(annotation_frame, name="My Image")


labels_layer = viewer.add_labels(np.zeros_like(annotation_frame).astype(int), seed = 0)
viewer.window.add_dock_widget(clf_widget)
prediction_layer = viewer.add_image(np.zeros_like(annotation_frame),colormap='PiYG')
prediction_layer.visible = False
prediction_layer.name = "Classifier prediction"
napari.run()

/Users/max/opt/anaconda3/lib/python3.7/site-packages/napari/_qt/__init__.py:54: UserWarning: 

IMPORTANT:
You are using QT version 5.12.1, but version 5.12 was also found in your environment.
This usually happens when you 'conda install' something that also depends on PyQt
*after* you have pip installed napari (such as jupyter notebook).
You will likely run into problems and should create a fresh environment.
If you want to install conda packages into the same environment as napari,
please add conda-forge to your channels: https://conda-forge.org

  warn(message=warn_message)


In [8]:
## Create the required folders 
def create_folders(path, directory):
    for i in directory:
        iDir=os.path.join(path, i)
        try:
            if not os.path.exists(iDir):
                os.makedirs(iDir)
                print('Created folder: ', iDir)
            else: 
                print('Folder: ', iDir, 'already exists')
        except OSError:
            print ('Error: Creating directory. ' + iDir)
required_folders = ['mask', 'stim_mask'] #leave this

create_folders(output_folder_segmentation,required_folders)  #leave this

Folder:  /Volumes/imaging.data/Max/REF52/DLC_1/FRET/FRET_11_late/cp.out1/output/mask already exists
Folder:  /Volumes/imaging.data/Max/REF52/DLC_1/FRET/FRET_11_late/cp.out1/output/stim_mask already exists


### Run the segmentation on all images

In [9]:
shapes = [(512,512),(64,64),(32,32)]
models = init_VGG16_pyramid(shapes)
frame_segmented = frame_to_labels(raw)
for i, filename in enumerate(filenames):
    if 'FRET' in filename.split('/')[-1]:
        frame = io.imread(filename)
        print(filename)
        
        # if input image has multiple channels, select the correct one here:
        #frame = frame[channel_nb,:,:] #where channel_nb is the channel you want to segment on
        frame_segmented = frame_to_labels(frame)
        frame_mask, table = spot_mask_from_labels(frame_segmented)
        if i == 0:
            plt.imshow(frame_segmented)
            plt.show()
            plt.imshow(frame_mask)
            plt.show()
            break
        io.imsave(os.path.join(output_folder_segmentation,'stim_mask', os.path.basename(filename).split('.')[0]+ 'stimmask' +'.tiff'), frame_mask, check_contrast=False)
        io.imsave(os.path.join(output_folder_segmentation, 'mask', os.path.basename(filename).split('.')[0] + 'mask'+'.tiff'), frame_segmented, check_contrast=False)

/Volumes/imaging.data/Max/REF52/DLC_1/FRET/FRET_11_late/1B2_10_w26TIRFFRETacceptor.TIF
/Volumes/imaging.data/Max/REF52/DLC_1/FRET/FRET_11_late/1B2_11_w26TIRFFRETacceptor.TIF
/Volumes/imaging.data/Max/REF52/DLC_1/FRET/FRET_11_late/1B2_12_w26TIRFFRETacceptor.TIF
/Volumes/imaging.data/Max/REF52/DLC_1/FRET/FRET_11_late/1B2_13_w26TIRFFRETacceptor.TIF
/Volumes/imaging.data/Max/REF52/DLC_1/FRET/FRET_11_late/1B2_14_w26TIRFFRETacceptor.TIF
/Volumes/imaging.data/Max/REF52/DLC_1/FRET/FRET_11_late/1B2_15_w26TIRFFRETacceptor.TIF
/Volumes/imaging.data/Max/REF52/DLC_1/FRET/FRET_11_late/1B2_16_w26TIRFFRETacceptor.TIF
/Volumes/imaging.data/Max/REF52/DLC_1/FRET/FRET_11_late/1B2_17_w26TIRFFRETacceptor.TIF
/Volumes/imaging.data/Max/REF52/DLC_1/FRET/FRET_11_late/1B2_18_w26TIRFFRETacceptor.TIF
/Volumes/imaging.data/Max/REF52/DLC_1/FRET/FRET_11_late/1B2_19_w26TIRFFRETacceptor.TIF
/Volumes/imaging.data/Max/REF52/DLC_1/FRET/FRET_11_late/1B2_1_w26TIRFFRETacceptor.TIF
/Volumes/imaging.data/Max/REF52/DLC_1/FRET/F

/Volumes/imaging.data/Max/REF52/DLC_1/FRET/FRET_11_late/REF52_44_w26TIRFFRETacceptor.TIF
/Volumes/imaging.data/Max/REF52/DLC_1/FRET/FRET_11_late/REF52_45_w26TIRFFRETacceptor.TIF
/Volumes/imaging.data/Max/REF52/DLC_1/FRET/FRET_11_late/REF52_46_w26TIRFFRETacceptor.TIF
/Volumes/imaging.data/Max/REF52/DLC_1/FRET/FRET_11_late/REF52_47_w26TIRFFRETacceptor.TIF
/Volumes/imaging.data/Max/REF52/DLC_1/FRET/FRET_11_late/REF52_48_w26TIRFFRETacceptor.TIF
/Volumes/imaging.data/Max/REF52/DLC_1/FRET/FRET_11_late/REF52_49_w26TIRFFRETacceptor.TIF
/Volumes/imaging.data/Max/REF52/DLC_1/FRET/FRET_11_late/REF52_4_w26TIRFFRETacceptor.TIF
/Volumes/imaging.data/Max/REF52/DLC_1/FRET/FRET_11_late/REF52_50_w26TIRFFRETacceptor.TIF
/Volumes/imaging.data/Max/REF52/DLC_1/FRET/FRET_11_late/REF52_5_w26TIRFFRETacceptor.TIF
/Volumes/imaging.data/Max/REF52/DLC_1/FRET/FRET_11_late/REF52_6_w26TIRFFRETacceptor.TIF
/Volumes/imaging.data/Max/REF52/DLC_1/FRET/FRET_11_late/REF52_7_w26TIRFFRETacceptor.TIF
/Volumes/imaging.data/Max